<a href="https://colab.research.google.com/github/kimsojeong1225/DeepLearning/blob/master/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
'''VGG16 model for Keras.

# Reference:

- [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

'''
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs


WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'




In [0]:
"""Instantiates the VGG16 architecture.
VGG16 B 모델 구현

    Optionally loads weights pre-trainedon ImageNet. 
    Note that when using TensorFlow, for best performance you should set
    `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.

    The model and the weights are compatible(호환가능) with both TensorFlow and Theano. 
    The data formatconvention used by the model is the one specified in your Keras config file.
    -모델에 사용되는 format rule?이 keras config file에 명시되어있다 -> 확인가능한지, keras config file에서 정확히 뭘 할 수 있는지 알아보기

    # Arguments
        include_top: whether to include the 3 fully-connected layers at the top of the network.
        네트워크 윗부분에 3 fc layers 포함시킬건지
        weights: one of `None` (random initialization) or "imagenet" (pre-training on ImageNet).
        가중치 랜덤 초기화할건지('None') , 이미지넷에서 사전 학습된 가중치 사용할건지 ('imagenet')
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`) to use as image input for the model.
        모델에 들어갈 input 이미지로 사용될 케라스 텐서
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape has to be `(224, 224, 3)` (with `channels_last` data format)  or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels, and width and height should be no smaller than 48. E.g. `(200, 200, 3)` would be one valid value.
        optional인데 include_top변수가 False일 경우에는 꼭 input shape를 명시해줘야함 : 채널은 무조건3(필터가 3x3이라서) , 넓이/높이는 48이상(왜?)
        pooling: Optional pooling mode for feature extraction when `include_top` is `False`.
        'include_top'이 False일 때 특성추출에 사용할 풀링 사용여부
            - `None` means that the output of the model will be the 4D tensor output of the last convolutional layer.
            'None': output 4D tensor
            - `avg` means that global average pooling will be applied to the output of the last convolutional layer, and thus the output of the model will be a 2D tensor.
            'avg' : average pooling -> output :2D tensor
            - `max` means that global max pooling will be applied.
            'max' : global max pooling 사용
        classes: optional number of classes to classify images into, only to be specified if `include_top` is True, and if no `weights` argument is specified.
        classes : optional값 몇개의 클래스로 이미지를 분류할건지 'include_top'=True이고 'weights'은 명시하지 않은 경우엔 꼭 명시하기

    # Returns
        A Keras model instance.

    # Raises : input shape , weights 변수 값 이상할 때 오류 나게 함
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """

In [0]:
def VGG16(include_top=True, weights='imagenet',
          input_tensor=None, input_shape=None,
          pooling=None,
          classes=1000):
    
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48, //vgg net 첫번째 layer 필터 수 때문에
                                      data_format=K.image_data_format(),
                                      include_top=include_top)

#Input Layer 만들어주는 코드
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='block5_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model



In [0]:
"""
스크립트 파일을 직접 실행할 경우에만 코드를 수행되게 만들려면..
if __name__ == __main__: 조건 아래에 들여쓰기를 해서 작성하면 되는 것이다. 
이와 같은 코드를 통해 C언어나 Java 에서와 같은 main() 함수의 역할을 할 수 있게 된다. 
"""
if __name__ == '__main__':
    model = VGG16(include_top=True, weights='imagenet')
    img_path = 'elephant.jpg'
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    print('Input image shape:', x.shape)

    preds = model.predict(x)
    print('Predicted:', decode_predictions(preds))